So now, let’s load the data.

In [18]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader 
# from torchnlp.encoders import label_encoder
import torch


Now we import the Dataset module to inherit various functions such as __getitem__(), __len__(), etc predefined in the library. These functions would help us to create our custom class for initializing the dataset. The code below shows how to create a dataset class.

In [19]:
class TitanicDataset(Dataset):
    def __init__(self,csvpath, mode = 'train'):
        self.mode = mode
        df = pd.read_csv(csvpath)
#         le = LabelEncoder()        
#       """       
#         <------Some Data Preprocessing---------->
#         Removing Null Values, Outliers and Encoding the categorical labels etc
#       """
        if self.mode == 'train':
            df = df.dropna()
            self.inp = df.iloc[:,1:].values
            self.oup = df.iloc[:,0].values.reshape(183,)
        else:
            self.inp = df.values
    def __len__(self):
        return len(self.inp)
    def __getitem__(self, idx):
        if self.mode == 'train':
            inpt  = torch.Tensor(self.inp[idx])
            oupt  = torch.Tensor(self.oup[idx])
            return { 'inp': inpt,
                     'oup': oupt,
            }
        else:
            inpt = torch.Tensor(self.inp[idx])
            return { 'inp': inpt
            }

Note: In the above code the last column of our data frame contains the target class while rest are input features hence we split it out to self.inp and self.oup variables accordingly and we would need both inputs as well as output if we are going to train else only the input data would be needed.

The __init__() function reads the .csv file using the pandas data frame and we do some preprocessing on it later (which is irrelevant to this tutorial). 

The __len__ ()function returns the number of examples and __getitem__() is used to fetch data by using its index. The important thing to note from the above piece of code is that we have converted our training examples into a tensor using the torch.tensor function while calling it using its index. So throughout the tutorial wherever we fetch examples it will all be in the form of tensors.
Now since the data is ready let’s load it into batches. This can be done easily using the DataLoader function as below.

In [21]:
## Initialize the DataSet
BATCH_SIZE = 1
data = TitanicDataset('C:/Users/hxi00/Documents/Udacity_AI4T/TitanicProj/data/train.csv')
## Load the Dataset
data_train = DataLoader(dataset = data, batch_size = BATCH_SIZE, shuffle =False)

You have to pass your dataset object resulting from the previous function as your argument. According to the number of batches, the result will be a multidimensional tensor of the shape (no_of_batches, batch_size, size_of_the_vector). Now, the number of dimensions would vary for other kinds of data like Image or Sequential Data accordingly based on its nature. But for now, just understand that there are multiple batches and each batch contains some examples equal to batch size (Irrespective of whatever data you use).

# Neural Network Architecture
Now since we have our data ready for training we have to design the neural network before we can start training it. Any model with conventionally used hyperparameters would be fine (Adam Optimizer, MSE Loss). To code our neural network, we can make use of the nn.Module to create the same

In [25]:
import torch.nn as nn
import torch.nn.functional as F

def swish(x):
    return x * F.sigmoid(x)

class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(8, 16)
        self.b1 = nn.BatchNorm1d(16)
        self.fc2 = nn.Linear(16, 8)
        self.b2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8,4)
        self.b3 = nn.BatchNorm1d(4)
        self.fc4 = nn.Linear(4,1)

    def forward(self,x):

#         x = swish(self.fc1(x))
#         x = self.b1(x)
#         x = swish(self.fc2(x))
#         x = self.b2(x)
#         x = swish(self.fc3(x))
#         x = self.b3(x)
#         x = F.sigmoid(self.fc4(x))
        
        x = F.relu(self.fc1(x))
        x = self.b1(x)
        x = F.relu(self.fc2(x))
        x = self.b2(x)
        x = F.relu(self.fc3(x))
        x = self.b3(x)
        x = F.sigmoid(self.fc4(x))

        return x

nn.Linear(), nn.BatchNorm1d() all become available once you inherit nn.Module class(). You can then simply use them by calling it. Since we are using simple tabular data we can use a simple dense layer (or fully connected layer) to create the model. For activation, I have used swish() by a custom definition. One could go for ReLu also. ReLu is available in the nn.functional() module. You could simply replace swish() with F.relu(). Since its a binary classification it is not very necessary to use a softmax in the final layer. I have used the sigmoid function to classify my examples. In the above code, __init__() helps you to initialize your neural network model as soon as you call the constructor and forward() function controls the data flow through the network which makes it responsible for feedforward. As we proceed to the training loop you will see how we call the forward function.

# Training the Model
Your training process can be laid as follow:

You define your training parameters like no of epochs, loss function, optimizer. All the optimizers are available in torch.optim(). Your optimizer function takes the weights of the network as its parameters. In the below code net variable contains the neural network model we created in the above subsection and net.parameters() refer to the network’s weights.

In [27]:
from torch.optim import Adam
net_parameters = [0.5, 0.5]
criterion = nn.MSELoss()
EPOCHS = 200
optm = Adam(net_parameters, lr = 0.001)

TypeError: optimizer can only optimize Tensors, but one of the params is float